## Instructions of using this notebook

### Steps:
1. Comment out line 155 (shutil.rmtree(detction_output_path) in *SEED-Event-Object-Detection/Model/detection.py* to save the detection results file.
2. copy the saved detection results directory to *SEED-Event-Object-Detection/utils/different_ratiolabel_validate/dt_results* or *dt_results_convnn* directory.
3. rename the copied directory to *Gen4_raw*
4. Change the variable *threshold_up* and *threshold_down* in pair [0.01, 0], [0.05, 0.01], [0.1, 0.05], [0.5, 0.1] and [1.0, 0.5]. Change the detction_output_path for each change.
5. Run the notebook for 5 times to get the evaluation results of different events density.


In [1]:
import sys
sys.path.append('/home/shenqi/Master_thesis/SEED')
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from metavision_ml.detection.anchors import Anchors

from Model.feature_extractor import  EMinGRU_ReLUFuseDownsampleConv_EventConv
from Model.ssd_head import BoxHead
from Model.detection import inference_step
from Model.detection import evaluate
from utils.dataloader import seq_dataloader
from utils.PropheseeToolbox.src.metrics.coco_eval import evaluate_detection
from utils.PropheseeToolbox.src.io.box_loading import reformat_boxes
from utils.PropheseeToolbox.src.io.box_filtering import filter_boxes
from metavision_sdk_core import EventBbox
from collections import defaultdict
import glob


dataset_path = '/media/shenqi/data/Gen4_multi_timesurface_FromDat'
dataset_type = 'gen4'
dataloader = seq_dataloader(dataset_path = dataset_path, dataset_type = dataset_type, num_tbins = 1, batch_size = 24, channels = 6)


In [ ]:
threshold_up = 1
threshold_down = 0.5
detction_output_path ='./dt_results_convnn/Gen4_1/'
output_dt_delete_list = [[] for i in range(dataloader.batch_size)]
output_gt_delete_list = [[] for i in range(dataloader.batch_size)]
with tqdm(total=len(dataloader.seq_dataloader_test), desc=f'Validation',ncols=120) as pbar:
    cnt = [0] * dataloader.batch_size      
    for data in dataloader.seq_dataloader_test:
        pbar.update(1)
        batch = data["inputs"]
        for index, video_info in enumerate(data['video_infos']):
            cnt[index] += 1
            file_root_path = str(video_info[0]).split(' ')[2].split('\n')[0].split('.')[0].split('/')[-1]
            dt_file_path = './dt_results_convnn/Gen4_raw/' + file_root_path + '_dt_bbox.npy'
            gt_file_path = './dt_results_convnn/Gen4_raw/' + file_root_path + '_GroundTruth_bbox.npy'
            dt_file = np.load(dt_file_path)
            gt_file = np.load(gt_file_path)
            
            dt_box_interval = dt_file[ (dt_file['t'] >= (cnt[index]-1) * 50000) & (dt_file['t'] < cnt[index]*50000)]
            gt_box_interval = gt_file[ (gt_file['t'] >= (cnt[index]-1) * 50000) & (gt_file['t'] < cnt[index]*50000)]
            
            origin_inputs = batch[0][index]
            
            for dt_box_events in dt_box_interval:
                dt_topleft_x = np.clip(dt_box_events["x"], 0, dataloader.width - 1).astype('int')
                dt_topleft_y = np.clip(dt_box_events["y"], 0, dataloader.height - 1).astype('int')
                dt_botright_x = np.clip(dt_box_events["x"] + dt_box_events["w"], 0, dataloader.width - 1).astype('int')
                dt_botright_y = np.clip(dt_box_events["y"] + dt_box_events["h"], 0, dataloader.height - 1).astype('int')
                
                dt_event_ave = torch.where(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x] == 0, torch.zeros_like(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x]),
                                        torch.ones_like(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x])).mean()
                if dt_event_ave < threshold_up and dt_event_ave > threshold_down:
                    output_dt_delete_list[index].append(dt_box_events)
            
            for gt_box_events in gt_box_interval:
                gt_topleft_x = np.clip(gt_box_events["x"], 0, dataloader.width - 1).astype('int')
                gt_topleft_y = np.clip(gt_box_events["y"], 0, dataloader.height - 1).astype('int')
                gt_botright_x = np.clip(gt_box_events["x"] + gt_box_events["w"], 0, dataloader.width - 1).astype('int')
                gt_botright_y = np.clip(gt_box_events["y"] + gt_box_events["h"], 0, dataloader.height - 1).astype('int')
                
                gt_event_ave = torch.where(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x] == 0, torch.zeros_like(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x]),
                                        torch.ones_like(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x])).mean()
                if gt_event_ave < threshold_up and gt_event_ave > threshold_down:
                    output_gt_delete_list[index].append(gt_box_events)
            


            if cnt[index] == 1200:
                dt_file_name_path = detction_output_path + file_root_path + '_dt_bbox.npy'
                np.save(dt_file_name_path, np.array(output_dt_delete_list[index], dtype = EventBbox)) 
                cnt[index] = 0
                output_dt_delete_list[index] = []
                
                gt_file_name_path = detction_output_path + file_root_path + '_GroundTruth_bbox.npy'
                np.save(gt_file_name_path, np.array(output_gt_delete_list[index], dtype = EventBbox)) 
                cnt[index] = 0
                output_gt_delete_list[index] = []
            


In [2]:

dt_detections = defaultdict(list)
gt_detections = defaultdict(list)



test_gt_files = sorted(glob.glob(detction_output_path + '/*_GroundTruth_bbox.npy'))   
gt_boxes_list = [np.load(p) for p in test_gt_files]
test_dt_files = sorted(glob.glob(detction_output_path + '/*_dt_bbox.npy'))  
result_boxes_list = [np.load(p) for p in test_dt_files]

gt_boxes_list = [reformat_boxes(p) for p in gt_boxes_list]
result_boxes_list = [reformat_boxes(p) for p in result_boxes_list]


min_box_diag,min_box_side = 60, 20
filter_boxes_fn = lambda x:filter_boxes(x, 500000, min_box_diag, min_box_side)
gt_boxes_list = map(filter_boxes_fn, gt_boxes_list)
result_boxes_list = map(filter_boxes_fn, result_boxes_list)

evaluate_detection(gt_boxes_list, result_boxes_list,classes=['background'] + dataloader.wanted_keys, height=dataloader.height, width=dataloader.width,time_tol=40000)



creating index...
index created!
Loading and preparing results...
DONE (t=0.34s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.82s).
Accumulating evaluation results...
DONE (t=1.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.249
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.455
 Average Recall     (AR) @[ IoU=0.

In [ ]:
'''
EMGU

0.01:
creating index...
index created!
Loading and preparing results...
DONE (t=1.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=24.98s).
Accumulating evaluation results...
DONE (t=5.64s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.336
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.210
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.214
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.323
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.361
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.392
 
 0.05:
 index created!
Loading and preparing results...
DONE (t=0.59s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.91s).
Accumulating evaluation results...
DONE (t=2.69s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.418
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.533
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.539
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.505
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.542
 
 0.1:
 creating index...
index created!
Loading and preparing results...
DONE (t=0.47s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.99s).
Accumulating evaluation results...
DONE (t=2.69s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.454
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.713
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.494
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.497
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.451
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.585
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.586
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.582
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.593
 
 0.5:
 creating index...
index created!
Loading and preparing results...
DONE (t=4.83s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=86.07s).
Accumulating evaluation results...
DONE (t=22.41s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.516
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.839
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.584
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.388
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.661
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.674
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.663
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.701
 
 1:
 creating index...
index created!
Loading and preparing results...
DONE (t=0.46s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.29s).
Accumulating evaluation results...
DONE (t=1.36s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.444
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.297
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.509
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.471
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.611
 
'''

'''
conv nn
0.01:
creating index...
index created!
Loading and preparing results...
DONE (t=1.72s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=40.88s).
Accumulating evaluation results...
DONE (t=7.66s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.157
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.082
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.257
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.181
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.292
 
 0.05
 creating index...
index created!
Loading and preparing results...
DONE (t=0.71s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.79s).
Accumulating evaluation results...
DONE (t=3.87s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.295
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.285
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.242
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.317
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.462
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.489
 
 0.1:
 creating index...
index created!
Loading and preparing results...
DONE (t=0.59s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.52s).
Accumulating evaluation results...
DONE (t=3.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.418
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.386
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.432
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.546
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.560
 
 0.5:
 creating index...
index created!
Loading and preparing results...
DONE (t=4.15s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=91.41s).
Accumulating evaluation results...
DONE (t=26.85s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.494
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.455
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.542
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.366
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.627
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.645
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.636
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.668
 
 1:
 creating index...
index created!
Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.90s).
Accumulating evaluation results...
DONE (t=1.24s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.249
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.553

'''

'\nconv nn\n0.01:\ncreating index...\nindex created!\nLoading and preparing results...\nDONE (t=1.72s)\ncreating index...\nindex created!\nRunning per image evaluation...\nEvaluate annotation type *bbox*\nDONE (t=40.88s).\nAccumulating evaluation results...\nDONE (t=7.66s).\n Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.065\n Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.157\n Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.045\n Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000\n Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.039\n Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.082\n Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.125\n Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.216\n Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0